In [1]:
#IMPORT IMPORTANCE LIBRARY
import pandas as pd
import numpy as np
import re
import ast

In [11]:
df = pd.read_csv('C:/Users/admin/Documents/Data Science/Project/BOOK_RECOMMENDER/1. crawling_data/raw_book_data.csv')

In [30]:
#FUNCTION TO SEE INFO OF COLS IN DATAFRAME
def info_df(df):
    info = []
    for i in df.columns:
        info.append([i, df[i].dtypes, df[i].nunique(), df[i].isna().sum(), round(100 * df[i].isna().sum()/ len(df[i]),2)])
    return pd.DataFrame(info, columns = ['col', 'dtype', 'nunique', 'na', 'na_percent (%)']).sort_values(by = 'dtype')

#FUNCTION TO PROCESS DATA IN DATAFRAME
def take_info_cate(i, idx):
    def str_to_list(i):
        l = re.findall(r"'(.*?)'", i)
        return l
    l = str_to_list(i)
    info = l[idx]
    return info
def take_info_book(x, label):
    def to_dict(x):
        try:
            x = dict(ast.literal_eval(x))
        except ValueError:
            l = ast.literal_eval(x)
            x = []
            for i in l:
                if len(i) > 2:
                    i = {i[0]: i[1:]}
            x.append(i)
            x = dict(x)
        return x
    x = to_dict(x)
    data = np.nan
    for k, v in x.items():
        if k == label:
            data = v
    return data
def take_rating(i , idx):
    try:
        if idx == 0:
            i = float(i.split('\n')[idx])
        elif idx == 1:
            i = i.split('\n')[idx]
            i = re.findall(r'\d', i)
            i = float(''.join(i))  
    except:
        i = np.nan
    return i
def take_num(str):
    try:
        if 'tr+' in str:
            num = 1000000 * float(''.join(re.findall(r'\d', str)))
        elif 'k+' in str:
            num = 1000 * float(''.join(re.findall(r'\d', str)))
        else:
            num = float(''.join(re.findall(r'\d', str)))
    except:
        num = np.nan
    return num
def float_discount(str):
    try:
        str = -1 * float(str.replace('%', '')) / 100
    except:
        str = np.nan
    return str
def take_measurements(x, take_type):
    def to_list(x):
        results = []
        if not isinstance(x, float):
            x = x.replace('cm','')
            x = x.split('x')
            for i in x:
                try:
                    i = float(i.replace(',','.'))
                except:
                    i = np.nan
                results.append(i)
        else:
            results = np.nan
        return results
    results = to_list(x)
    try:
        if take_type == 'width':
            results = results[0]
        elif take_type == 'length':
            results = results[1]
    except:
        results = np.nan

    return results

In [31]:
info_df(df)

,col,dtype,nunique,na,na_percent (%)
0,Unnamed: 0,int64,215,0,0.00
10,seller_star,float64,8,0,0.00
1,category,object,1968,0,0.00
2,img,object,1973,0,0.00
3,price,object,729,0,0.00
4,discount,object,56,177,8.96
5,sale_quantities,object,1034,19,0.96
6,rating,object,927,141,7.14
7,info,object,1886,0,0.00
8,describe,object,1944,0,0.00


In [38]:
#PROCESS DATA TO WELL FORM
df['book_name'] = df['category'].apply(take_info_cate, args = (-1,))
df['detail_cate'] = df['category'].apply(take_info_cate, args = (-2,))
df['large_cate'] = df['category'].apply(take_info_cate, args = (-3,))

df['discount'] = df['discount'].map(float_discount)

prod_info_col = ['Hàng chính hãng', 'Công ty phát hành', 'Ngày xuất bản', 'Kích thước', 'Loại bìa', 'Số trang', 'Nhà xuất bản']
for i in prod_info_col:
    df[i] = df['info'].apply(take_info_book, args = (i,))

df['rating star'] = df['rating'].apply(take_rating, args = (0,))
df['rating_quantity'] = df['rating'].apply(take_rating, args = (1,))

df['book_length'] = df['Kích thước'].apply(take_measurements, args = ('length', ))
df['book_width'] = df['Kích thước'].apply(take_measurements, args = ('width', ))

for i in ['price', 'sale_quantities', 'seller_reviews_quantity', 'seller_follow', 'Số trang']:
    df[i] = df[i].map(take_num)

df['Ngày xuất bản'] = pd.to_datetime(df['Ngày xuất bản'])

In [53]:
#RENAME COLS
rename = {'Hàng chính hãng': 'authentic', 'Công ty phát hành': 'publishing_company', 'Ngày xuất bản': 'publish_date', 'Loại bìa': 'cover_type', 'Số trang': 'num_of_pages', 'Nhà xuất bản': 'publisher'}
df = df.rename(rename, axis = 1)
info_df(df)

,col,dtype,nunique,na,na_percent (%)
21,book_width,float64,44,459,23.24
1,price,float64,729,0,0.00
2,discount,float64,56,177,8.96
3,sale_quantities,float64,1034,19,0.96
19,rating_quantity,float64,630,141,7.14
18,rating star,float64,16,141,7.14
6,seller_star,float64,8,0,0.00
7,seller_reviews_quantity,float64,94,0,0.00
8,seller_follow,float64,101,0,0.00
20,book_length,float64,46,458,23.19


In [56]:
#SAVE DATAFRAME TO CSV
df.to_csv('wrangled_book_data.csv')